# Exercise sheet 4

In this exercise sheet we will code all components needed to build and train a (deep) neural network. There will be multiple functions to implement, most of which are followed by a verification block that allows you to check if your function works as expected.

**A note on the used notation**:

- Superscript $[l]$ denotes a quantity associated with the $l^{th}$ layer. 
    - Example: $a^{[L]}$ is the $L^{th}$ layer activation. $W^{[L]}$ and $b^{[L]}$ are the $L^{th}$ layer parameters.
- Superscript $(i)$ denotes a quantity associated with the $i^{th}$ example. 
    - Example: $x^{(i)}$ is the $i^{th}$ training example.
- Lowerscript $i$ denotes the $i^{th}$ entry of a vector.
    - Example: $a^{[l]}_i$ denotes the $i^{th}$ entry of the $l^{th}$ layer's activations).


We start with loading some standard libraries.

In [3]:
# plt to plot figures
import matplotlib.pyplot as plt
# numpy for array/matrix operations
import numpy as np
# to be able to load matlab data files
from scipy.io import loadmat

# we set a seed variable for functions that use randomization (e.g. when shuffling data samples)
# this way, we can have reproducible results even with randomization
RANDOM_STATE = 2
np.random.seed(RANDOM_STATE)

## 1) Outline

To build your neural network, you will be implementing several "helper functions". These helper functions can be used to build a L-layer neural network. Each small helper function you will implement will have instructions that will walk you through the necessary steps. Here is an outline of this assignment, you will:

- Initialize the parameters for an L-layer neural network.
- Implement the forward propagation module (shown in purple in the figure below).
    - Complete the LINEAR part of a layer's forward propagation step (resulting in output Z).
    - Implement activation functions (relu/sigmoid)
    - Combine the previous two steps into a new [LINEAR->ACTIVATION] forward function.
    - Stack the [LINEAR->RELU] forward function L-1 time (for layers 1 through L-1) and add a [LINEAR->SIGMOID] at the end (for the final layer L). This gives you a new L_model_forward function.
- Compute the loss.
- Implement the backward propagation module (denoted in red in the figure below).

    - Complete the LINEAR part of a layer's backward propagation step.
    - Functions for calculating the gradient of the ACTIVATION functions are given (relu_backward/sigmoid_backward)
    - Combine the previous two steps into a new [LINEAR->ACTIVATION] backward function.
    - Stack [LINEAR->RELU] backward L-1 times and add [LINEAR->SIGMOID] backward in a new L_model_backward function

- Finally update the parameters using gradient descent.

Note that for every forward function, there is a corresponding backward function. That is why at every step of your forward module you will be storing some values in a cache. The cached values are useful for computing gradients. In the backpropagation module you will then use the cache to calculate the gradients. This assignment will show you exactly how to carry out each of these steps.


## 2) Initialization of L-layer Neural Network

Complete implementing the function below to randomly initialize parameters (weight matrices and bias vectors) for a (deep) L-layer neural network. Make sure that dimensions of the parameter matrices/vectors match between each layer (assert calls for this are already implemented).

Notes:
- Initialize weight matrices with small normal distributed numbers. Use np.random.randn(matrix_shape) * 0.01.
- Initialize bias vectors with zeros. Use np.zeros(vector_shape).

In [4]:
def initialize_parameters(layer_dims):
    """
    Argument:
    layer_dims -- python array (list) containing the dimensions of each layer in our network:
                    e.g. a network with input layer (e.g. 4 features), two hidden layers (e.g. each with 5 nodes)
                    and output layer (e.g. 1 node for binary classification using sigmoid activation) would result 
                    in layer_dims=[4,5,5,1]
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
                    
    """
    np.random.seed(RANDOM_STATE)
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        
        # initializing weight matrix 'W' with random values (to introduce diversity in the learning process)
        # Without random initialization, all the neurons in a layer would compute the same gradients and update
        # their weights in the same way. This limits the capacity of the network to learn diverse features!
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        
        # initializing bias vector 'b' with zeros
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        ### END CODE HERE ###

        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

**Verify that code works as expected**

Compare output of your implemented function with expected output below.

In [5]:
example_layer_dims = [4,5,5,1]

parameters = initialize_parameters(example_layer_dims)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[-4.16757847e-03 -5.62668272e-04 -2.13619610e-02  1.64027081e-02]
 [-1.79343559e-02 -8.41747366e-03  5.02881417e-03 -1.24528809e-02]
 [-1.05795222e-02 -9.09007615e-03  5.51454045e-03  2.29220801e-02]
 [ 4.15393930e-04 -1.11792545e-02  5.39058321e-03 -5.96159700e-03]
 [-1.91304965e-04  1.17500122e-02 -7.47870949e-03  9.02525097e-05]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-8.78107893e-03 -1.56434170e-03  2.56570452e-03 -9.88779049e-03
  -3.38821966e-03]
 [-2.36184031e-03 -6.37655012e-03 -1.18761229e-02 -1.42121723e-02
  -1.53495196e-03]
 [-2.69056960e-03  2.23136679e-02 -2.43476758e-02  1.12726505e-03
   3.70444537e-03]
 [ 1.35963386e-02  5.01857207e-03 -8.44213704e-03  9.76147160e-08
   5.42352572e-03]
 [-3.13508197e-03  7.71011738e-03 -1.86809065e-02  1.73118467e-02
   1.46767801e-02]]
b2 = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


#### Expected output:

W1 = [[-4.16757847e-03 -5.62668272e-04 -2.13619610e-02  1.64027081e-02]
 [-1.79343559e-02 -8.41747366e-03  5.02881417e-03 -1.24528809e-02]
 [-1.05795222e-02 -9.09007615e-03  5.51454045e-03  2.29220801e-02]
 [ 4.15393930e-04 -1.11792545e-02  5.39058321e-03 -5.96159700e-03]
 [-1.91304965e-04  1.17500122e-02 -7.47870949e-03  9.02525097e-05]]
 
b1 = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]

W2 = [[-8.78107893e-03 -1.56434170e-03  2.56570452e-03 -9.88779049e-03
  -3.38821966e-03]
 [-2.36184031e-03 -6.37655012e-03 -1.18761229e-02 -1.42121723e-02
  -1.53495196e-03]
 [-2.69056960e-03  2.23136679e-02 -2.43476758e-02  1.12726505e-03
   3.70444537e-03]
 [ 1.35963386e-02  5.01857207e-03 -8.44213704e-03  9.76147160e-08
   5.42352572e-03]
 [-3.13508197e-03  7.71011738e-03 -1.86809065e-02  1.73118467e-02
   1.46767801e-02]]

b2 = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


## 3) Forward propagation

Now that we have initialized the parameters, we implement the forward propagation module. We start by implementing some basic functions that are needed later on. We'll complete three functions in this part:

    - LINEAR
    - LINEAR -> ACTIVATION where ACTIVATION will be either ReLU or Sigmoid.
    - [LINEAR -> RELU] x (L-1) -> LINEAR -> SIGMOID (i.e. whole model)

### 3.1) Linear Forward Function

We start with the linear forward function. This function calculates the equation:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$$

with $A^{[0]} = X$ (i.e. the matrix of data samples).

Remember that $W^{[l]}A^{[l-1]}$ calculates the dot product between two matrices.

In [6]:
def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    Z = np.dot(W, A) + b
    ### END CODE HERE ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

**Verify that code works as expected**

Compare output of your implemented function with expected output below.

In [7]:
np.random.seed(RANDOM_STATE)
example_batch_size = 1  # number of data samples in this batch

A = np.random.randn(example_layer_dims[0], example_batch_size)
W = np.random.randn(example_layer_dims[1], example_layer_dims[0])
b = np.random.randn(example_layer_dims[1], 1)

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[-2.66089417]
 [ 2.83770075]
 [-2.7214624 ]
 [ 0.36664977]
 [-3.21640702]]


#### Expected output: 

Z = [[-2.66089417]
 [ 2.83770075]
 [-2.7214624 ]
 [ 0.36664977]
 [-3.21640702]]

### 3.2) Activation Functions

There are various different activation functions. In this exercise, we implement two activation functions that were presented in the lecture: sigmoid and ReLU. We return the input (Z) as well (for simplification of error backpropagation calculations later on).

Note: the backward functions for both activation functions (i.e. derivative including application of chain rule - therefore not exactly the same as in the lecture) is already given.

In [8]:
# activation functions

def sigmoid(Z):
    ### START CODE HERE ### (≈ 1 line of code)
    A = 1 / (1 + np.exp(-Z))
    ### END CODE HERE ###
    return A, Z

def relu(Z):
    ### START CODE HERE ### (≈ 1 line of code)
    A = Z * (Z > 0)
    ### END CODE HERE ###
    assert(A.shape == Z.shape)
    return A, Z


# backward functions (includes chain rule application)

def sigmoid_backward(dA, Z):    
    # version from the lecture:
    dZ = dA * np.exp(-Z) / (1 + np.exp(-Z)) ** 2
    # alternative version
    #s = 1/(1+np.exp(-Z))
    #dZ = dA * s * (1 - s)
    assert (dZ.shape == Z.shape)
    return dZ

def relu_backward(dA, Z):
    dZ = np.zeros(dA.shape)
    dZ[Z > 0] = 1
    assert (dZ.shape == Z.shape)
    return dZ

### 3.3) Linear Activation Forward Function

One layer in a neural network usually consists of a linear function that is followed by an activation function (and thus the two combined are counted as one layer). For convenience, we group these two operations into one function (LINEAR->ACTIVATION). Implement a function that does the LINEAR forward step followed by an ACTIVATION forward step.

The mathematical formulation is given as: 
$A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]})$ 

where the activation "g" can be sigmoid() or relu(). Use linear_forward() from 3.1) and the correct activation function from 3.2) to implement the function below.

In [9]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python tuple containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        ### END CODE HERE ###
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

**Verify that code works as expected**

Compare output of your implemented function with expected output below.

In [10]:
np.random.seed(RANDOM_STATE)
example_batch_size = 1  # number of data samples in this batch

A_prev = np.random.randn(example_layer_dims[0], example_batch_size)
W = np.random.randn(example_layer_dims[1], example_layer_dims[0])
b = np.random.randn(example_layer_dims[1], 1) 

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))



With sigmoid: A = [[0.06532072]
 [0.94467943]
 [0.06171873]
 [0.5906492 ]
 [0.03855294]]
With ReLU: A = [[-0.        ]
 [ 2.83770075]
 [-0.        ]
 [ 0.36664977]
 [-0.        ]]


#### Expected output:
    
With sigmoid: A = [[0.06532072]
 [0.94467943]
 [0.06171873]
 [0.5906492 ]
 [0.03855294]]
 
With ReLU: A = [[0.        ]
 [2.83770075]
 [0.        ]
 [0.36664977]
 [0.        ]]


### 3.4) L-Layer Model

We can now use the implemented functions to build an L-layer neural network with a sigmoid activation function in the output layer. Write a function that creates a neural network with L-1 layers using linear_activation_forward with ReLU activation, then finishes with the output layer, using linear_activation_forward with sigmoid activation.

Note on counting layers: when counting layers in a neural network, we only count layers with learnable parameters (weight matrices / bias vectors). This is why the input layer is usually NOT counted.

In [11]:
def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L-1 of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2    # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        W = parameters["W" + str(l)]
        b = parameters["b" + str(l)]
        A, cache = linear_activation_forward(A_prev, W, b, activation="relu")
        caches.append(cache)
        ### END CODE HERE ###
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL, cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], activation="sigmoid")
    caches.append(cache)
    ### END CODE HERE ###
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

**Verify that code works as expected**

Compare output of your implemented function with expected output below.

In [12]:
example_layer_dims = [4,5,5,1]

np.random.seed(RANDOM_STATE)
example_batch_size = 1  # number of data samples in this batch

A_prev = np.random.randn(example_layer_dims[0], example_batch_size)
W = np.random.randn(example_layer_dims[1], example_layer_dims[0])
b = np.random.randn(example_layer_dims[1], 1)

X = A_prev
parameters = initialize_parameters(example_layer_dims)


AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))


AL = [[0.49999827]]
Length of caches list = 3


#### Expected output:

AL = [[0.49999827]]  
Length of caches list = 3

## 4) Loss function

So far we have implemented an L-layer neural network with full forward propagation, that takes input X (matrix of samples) and outputs a row vector containing the network's predictions. We also store all intermediate values that are calculated during forward propagation in "caches". Output of the neural net is given in $A^{[L]}$, which we can use to compute the loss of the net's predictions.

We will now implement the loss calculation. In this exercise, we use binary cross entropy loss, which is given as:

$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\ln\left(a^{[L] (i)}\right) + (1-y^{(i)})\ln\left(1- a^{[L](i)}\right))$$

with  
m = number of samples  
$a^{[L]}$ = output of the model   
y = ground truth class (0 or 1)

In [13]:
def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1] # number of samples

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    cost = -(1/m) * np.sum(Y * np.log(AL) + (1-Y) * np.log(1-AL))
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

**Verify that code works as expected**

Compare output of your implemented function with expected output below.

In [14]:
np.random.seed(RANDOM_STATE)

Y = np.array(np.random.randint(0,1, size=(1,1)))

#Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))



cost = 0.6931437185364475


#### Expected output:
    
cost = 0.6931437185364475

## 5) Backward propagation

Following the forward propagation of our network, we now need to implement helper functions for backpropagation. Remember that backpropagation is used to calculate the gradient of the loss function with respect to the parameters, so that we can update them using gradient descent and reduce the defined loss (and classification error as a consequence). These weight/bias updates represent learning of the network.

Similar to forward propagation, we build the backward propagation in three steps:

- LINEAR backward
- LINEAR -> ACTIVATION backward where ACTIVATION computes the derivative of either the ReLU or sigmoid activation
[LINEAR -> RELU]
- (L-1) -> LINEAR -> SIGMOID backward (whole model)

### 5.1) Linear backward function

For layer $l$, the linear (forward) function is given as: $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (followed by an activation).

Suppose we have already calculated the derivative $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$. We need to get $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ which are used by gradient descent to update the parameters $(W^{[l]}, b^{[l]})$.

$dW^{[l]}, db^{[l]}, dA^{[l-1]}$ are computed using $dZ^{[l]}$ as follows:

$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T}$$
$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]}$$

Use these formulas as reference to implement the function below.

In [24]:
def linear_backward(dZ, cache):
    # Here cache is "linear_cache" containing (A_prev, W, b) coming from the forward propagation in the current layer
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code); einfach von den Formeln übertragen
    dW = (1/m) * np.dot(dZ, A_prev.T) # .T ist die operation für Matrixtransposition
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ) # wieder matrix transposition
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

**Verify that code works as expected**

Compare output of your implemented function with expected output below.

In [25]:
example_layer_dims = [4,10,10,1]

np.random.seed(RANDOM_STATE)

#linear_cache = (A_prev, W, b)
linear_cache = caches[-1][0]

#Z, _ = linear_forward(A_prev, W, b)
Z, _ = linear_forward(linear_cache[0], linear_cache[1], linear_cache[2])

dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
dZ = sigmoid_backward(dAL, Z)

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[-0.00167838]
 [ 0.00305669]
 [ 0.00023985]
 [-0.00414566]
 [ 0.00043855]]
dW = [[0.         0.         0.         0.00041755 0.        ]]
db = [[0.49999827]]


#### Expected output:
    
dA_prev = [[-0.00167838]
 [ 0.00305669]
 [ 0.00023985]
 [-0.00414566]
 [ 0.00043855]]
 
dW = [[0.         0.         0.         0.00041755 0.        ]]

db = [[0.49999827]]


### 5.2) Linear activation backward function

Next, we'll implement a function that merges the backward step in linear_backward and the backward step for the activation function, i.e. we implement the backpropgation for a layer consisting of LINEAR->ACTIVATION. Note that we have already implemented the backward step (i.e. calculation of the derivative) for the activation functions sigmoid and ReLU with consideration of the chain rule used in backpropagation further above (called sigmoid_backward and relu_backward). 
Mathematically, what happens in those functions is that if $g(.)$ is the activation function, then 
    `sigmoid_backward` and `relu_backward` compute $$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]})$$
    
Use sigmoid_backward, relu_backward and linear_backward to complete the function below.  

In [26]:
def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 1 line of code)
        dZ = relu_backward(dA, activation_cache)
        ### END CODE HERE ###
        
    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 1 line of code)
        dZ = sigmoid_backward(dA, activation_cache)
        ### END CODE HERE ###
    
    ### START CODE HERE ### (≈ 1 line of code)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    ### END CODE HERE ###
    
    return dA_prev, dW, db

**Verify that code works as expected**

Compare output of your implemented function with expected output below.

In [27]:
linear_activation_cache = caches[-1]


dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

linear_activation_cache = caches[-2]

dA_prev, dW, db = linear_activation_backward(dA_prev, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))



sigmoid:
dA_prev = [[-0.00167838]
 [ 0.00305669]
 [ 0.00023985]
 [-0.00414566]
 [ 0.00043855]]
dW = [[0.         0.         0.         0.00041755 0.        ]]
db = [[0.49999827]]

relu:
dA_prev = [[ 1.35963386e-02]
 [ 5.01857207e-03]
 [-8.44213704e-03]
 [ 9.76147160e-08]
 [ 5.42352572e-03]]
dW = [[0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.07430675 0.         0.03073885 0.         0.01554262]
 [0.         0.         0.         0.         0.        ]]
db = [[0.]
 [0.]
 [0.]
 [1.]
 [0.]]


#### Expected output:


##### sigmoid:

sigmoid:
dA_prev = [[-0.00167838]
 [ 0.00305669]
 [ 0.00023985]
 [-0.00414566]
 [ 0.00043855]]
 
dW = [[0.         0.         0.         0.00041755 0.        ]]

db = [[0.49999827]]


##### relu:

dA_prev = [[ 1.35963386e-02]
 [ 5.01857207e-03]
 [-8.44213704e-03]
 [ 9.76147160e-08]
 [ 5.42352572e-03]]
 
dW = [[0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.07430675 0.         0.03073885 0.         0.01554262]
 [0.         0.         0.         0.         0.        ]]

db = [[0.]
 [0.]
 [0.]
 [1.]
 [0.]]

### 5.3) L-Layer Model Backward

We can now implement the loss backpropagation for the whole network. Recall that when we run the L_model_forward function, in each step we store a cache which contains (A,W,b, and Z). In the backpropagation module, we'll use those variables to compute the gradients. Therefore, in the L_model_backward function, we go through all the hidden layers backward, starting from layer L. In each step, we use the cached values for the l-th layer to backpropagate the error through that layer.

**Initializing backpropagation:** To backpropagate the loss through our network with final sigmoid activation in the output layer, we know that the output is given as: $A^{[L]} = \sigma(Z^{[L]})$
. Our code thus needs to compute $dAL= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$, where $\mathcal{L}$ is our loss function.

Thus, we need the partial derivative of our loss function with respect to AL, which is given by the following formula:

$dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))$

We can use this post-activation gradient dAL to keep going backward by feeding dAL into our LINEAR->SIGMOID backward function (which will use the cached values stored by the L_model_forward function). After that, we'll have to use a for loop to iterate through all the remaining layers using our LINEAR->RELU backward function. As output of the function, all calculated dA, dW, and db (i.e. for each layer) has to be stored in a python dict (called "grads", see below) using the naming conventions outlined below.

In [42]:
def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = -(np.divide(Y,AL) - np.divide(1-Y, 1-AL))
    ### END CODE HERE ###
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    ### END CODE HERE ###
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)], current_cache, activation="relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l+1)] = dW_temp
        grads["db" + str(l+1)] = db_temp
        ### END CODE HERE ###

    return grads


**Verify that code works as expected**

Compare output of your implemented function with expected output below.

In [43]:
grads = L_model_backward(AL, Y, caches)
print("dW1 = {}\ndb1 = {}\ndA1 = {}\n".format(grads['dW1'], grads['db1'], grads['dA1']))

dW1 = [[-0.41675785 -0.05626683 -2.1361961   1.64027081]
 [ 0.          0.          0.          0.        ]
 [-0.41675785 -0.05626683 -2.1361961   1.64027081]
 [ 0.          0.          0.          0.        ]
 [-0.41675785 -0.05626683 -2.1361961   1.64027081]]
db1 = [[1.]
 [0.]
 [1.]
 [0.]
 [1.]]
dA1 = [[ 1.35963386e-02]
 [ 5.01857207e-03]
 [-8.44213704e-03]
 [ 9.76147160e-08]
 [ 5.42352572e-03]]



#### Expected output:

dW1 = [[-0.41675785 -0.05626683 -2.1361961   1.64027081]
 [ 0.          0.          0.          0.        ]
 [-0.41675785 -0.05626683 -2.1361961   1.64027081]
 [ 0.          0.          0.          0.        ]
 [-0.41675785 -0.05626683 -2.1361961   1.64027081]]
 
db1 = [[1.]
 [0.]
 [1.]
 [0.]
 [1.]]
 
dA1 = [[ 1.35963386e-02]
 [ 5.01857207e-03]
 [-8.44213704e-03]
 [ 9.76147160e-08]
 [ 5.42352572e-03]]

### 5.4) Update parameters

We now have everything to update the parameters (weight matrices and bias vectors) of the model using gradient descent. These updates are essentially what constitutes "learning" in the neural network. We update weights and biases as follows:

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} $$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} $$

where $\alpha$ is the learning rate.

In [46]:
def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2  # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    for l in range (1, L+1):
        parameters["W" + str(l)] -= learning_rate * grads["dW" + str(l)]
        parameters["b" + str(l)] -= learning_rate * grads["db" + str(l)]

    ### END CODE HERE ###
    return parameters

**Verify that code works as expected**

Compare output of your implemented function with expected output below.

In [47]:
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))
print ("W3 = "+ str(parameters["W3"]))
print ("b3 = "+ str(parameters["b3"]))


W1 = [[ 0.03750821  0.00506401  0.19225765 -0.14762437]
 [-0.01793436 -0.00841747  0.00502881 -0.01245288]
 [ 0.03109626 -0.00346339  0.21913415 -0.141105  ]
 [ 0.00041539 -0.01117925  0.00539058 -0.0059616 ]
 [ 0.04148448  0.01737669  0.2061409  -0.16393683]]
b1 = [[-0.1]
 [ 0. ]
 [-0.1]
 [ 0. ]
 [-0.1]]
W2 = [[-8.78107893e-03 -1.56434170e-03  2.56570452e-03 -9.88779049e-03
  -3.38821966e-03]
 [-2.36184031e-03 -6.37655012e-03 -1.18761229e-02 -1.42121723e-02
  -1.53495196e-03]
 [-2.69056960e-03  2.23136679e-02 -2.43476758e-02  1.12726505e-03
   3.70444537e-03]
 [ 6.16566348e-03  5.01857207e-03 -1.15160218e-02  9.76147160e-08
   3.86926367e-03]
 [-3.13508197e-03  7.71011738e-03 -1.86809065e-02  1.73118467e-02
   1.46767801e-02]]
b2 = [[ 0. ]
 [ 0. ]
 [ 0. ]
 [-0.1]
 [ 0. ]]
W3 = [[-0.00335677  0.00611341  0.00047971 -0.00833311  0.0008771 ]]
b3 = [[-0.04999983]]


#### Expected output:

W1 = [[ 0.03750821  0.00506401  0.19225765 -0.14762437]
 [-0.01793436 -0.00841747  0.00502881 -0.01245288]
 [ 0.03109626 -0.00346339  0.21913415 -0.141105  ]
 [ 0.00041539 -0.01117925  0.00539058 -0.0059616 ]
 [ 0.04148448  0.01737669  0.2061409  -0.16393683]]
 
b1 = [[-0.1]
 [ 0. ]
 [-0.1]
 [ 0. ]
 [-0.1]]
 
W2 = [[-8.78107893e-03 -1.56434170e-03  2.56570452e-03 -9.88779049e-03
  -3.38821966e-03]
 [-2.36184031e-03 -6.37655012e-03 -1.18761229e-02 -1.42121723e-02
  -1.53495196e-03]
 [-2.69056960e-03  2.23136679e-02 -2.43476758e-02  1.12726505e-03
   3.70444537e-03]
 [ 6.16566348e-03  5.01857207e-03 -1.15160218e-02  9.76147160e-08
   3.86926367e-03]
 [-3.13508197e-03  7.71011738e-03 -1.86809065e-02  1.73118467e-02
   1.46767801e-02]]
   
b2 = [[ 0. ]
 [ 0. ]
 [ 0. ]
 [-0.1]
 [ 0. ]]
 
W3 = [[-0.00335677  0.00611341  0.00047971 -0.00833311  0.0008771 ]]

b3 = [[-0.04999983]]

## 6) Optional

We now have implemented all functions that are needed to build and train a deep neural network for a binary classification task. Try putting it all together and train a DNN for the last binary classification task on Exercise sheet 3 (data given in "ex3data2.mat").   



# 7) Q&A (Bonus Points)      Due: Fr, 15.07.22, 23:59

#### (T7) Your task: Answer the questions below and give explanations where required. 

- 7.1 ) Describe the components of a Multi Layer Perceptron.\
Input Layer - receives input data and passes it to the next layer. Number of neurons in input layer -> number of input features in the dataset.\
Hidden Layer - stand between input and output layer. they are responsible for learning the patterns within the data.\
Neuron - receives input, applies an activation function to get a value. The output is determined by the weighted sum of its inputs, passed through the activation function (only if they are in hidden layers or the output layer).\
Output Layer - produces the final prediction of the network. The number of neurons in the layer depends on the task of the network. For example, for binary classification, there might only be a single neuron.\
Weights and Biases - they form the connection between neurons. Each neuron has its own weights, which determine the strength of the connection. Also, each neuron has a bias value, which can shift the threshold of the activation function.

- 7.2 ) What do Neural Networks need to have to be able to estimate non-linear relationships in the data?\
Neural networks definitely need to have Activation Functions and a sizable training dataset to estimate non-linear relationships.

- 7.3 ) For a binary classification task on 5 input features; Imagine a deep Neural Network that consists of 3 hidden, fully connected layers with 1000, 500 and 1000 nodes respectively. How many total weights does the network have?\
5000 +  1000 * 500  +  500 × 1000  +  1000  =  1 006 000 weights

- 7.4 ) Describe a problem that large and very deep Neural Network architectures can have for small data sets.\
When dealing with a small dataset, a large and very deep NN has a higher risk of overfitting.

- 7.5 ) Describe how the prediction error for the training samples is used to estimate the weights of a deep Neural Network. Use the words "Loss function", "Back Propagation" and "Gradient Descent"\
The loss function measures the difference between the network's predictions and the ground truth of the training samples. Through backpropagation, the gradient of the loss function with respect to the network's weights is calculated. This helps us to determine how much each weight affects the overall cost. Then, we want to find the weights that minimize the loss function and reduce the cost. By using a technique called Gradient Descent, the weights get adjusted with the inverted gradient, and the NN iteratively minimizes the cost.  